<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/dolly_japanese_gpt_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dolly-japanese-gpt-1bをためす。
https://huggingface.co/inu-ai/dolly-japanese-gpt-1b

In [49]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [50]:
!pip install transformers sentencepiece
# restart runtime :)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("inu-ai/dolly-japanese-gpt-1b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("inu-ai/dolly-japanese-gpt-1b").to(device)

In [52]:
MAX_ASSISTANT_LENGTH = 100
MAX_INPUT_LENGTH = 1024
INPUT_PROMPT = r'<s>\n以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n[SEP]\n指示:\n{instruction}\n[SEP]\n入力:\n{input}\n[SEP]\n応答:\n'
NO_INPUT_PROMPT = r'<s>\n以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n[SEP]\n指示:\n{instruction}\n[SEP]\n応答:\n'

def prepare_input(instruction, input_text):
    if input_text != "":
        prompt = INPUT_PROMPT.format(instruction=instruction, input=input_text)
    else:
        prompt = NO_INPUT_PROMPT.format(instruction=instruction)
    return prompt

def format_output(output):
    output = output.lstrip("<s>").rstrip("</s>").replace("[SEP]", "").replace("\\n", "\n")
    return output

def generate_response(instruction, input_text):
    prompt = prepare_input(instruction, input_text)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            min_length=n,
            max_length=min(MAX_INPUT_LENGTH, n + MAX_ASSISTANT_LENGTH),
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bad_words_ids=[[tokenizer.unk_token_id]]
        )

    output = tokenizer.decode(output_ids.tolist()[0])
    formatted_output_all = format_output(output)
    response = f"Assistant:{formatted_output_all.split('応答:')[-1].strip()}"

    return formatted_output_all, response 

instruction = "あなたは何でも正確に答えられるAIです。"
questions = [
    "日本で一番高い山は？",
    "日本で一番広い湖は？",
    "世界で一番高い山は？",
    "世界で一番広い湖は？",
    "冗談を言ってください。",
]

# 各質問に対して応答を生成して表示
for question in questions:
    formatted_output_all, response = generate_response(instruction, question)
    print(response)


Assistant:富士山
Assistant:宍道湖
Assistant:エベレストは標高8,848メートルで、世界最高峰の山です。
Assistant:バイカル湖は、地球上で最も大きな淡水湖である。
Assistant:冗談は言わないでください、私はすべてを知っているのです。


In [56]:
instruction = "真実を述べます。"
questions = [
    "日本企業の将来は？",
    "好きなアニメと理由を１つだけ上げてください。",
    "世界で一番高い山と高さを教えてください。",
    "九州で訪問すべき観光地を教えてください。",
    "世界で一番面白い冗談を沢山考えてください。",
]

# 各質問に対して応答を生成して表示
for question in questions:
    formatted_output_all, response = generate_response(instruction, question)
    print("質問", question)
    print(response)

質問 日本企業の将来は？
Assistant:日本企業は、グローバルな競争にさらされ、技術力と生産性の向上、労働力の確保、より柔軟な労働環境、顧客への価値の向上、新しい市場への参入、そして顧客や市場の拡大を求められています。それに対応するために、日本企業は、グローバルなパートナーシップ、サプライヤー、および販売チャネルと協力する機会を模索しています。
質問 好きなアニメと理由を１つだけ上げてください。
Assistant:好きなアニメはたくさんありますが、中でも「Frozen」「God of War」「Infinity Blade」「The Shadow of Your Dragon」「Dune of the Dead」「Elder Scrolls V: Skyrim」「Days of Memories」「The Last of Us」「The December Slaughter: The Once Again」「Destiny of the World」「Fallout」「D
質問 世界で一番高い山と高さを教えてください。
Assistant:エベレストは9,704mの山です。
質問 九州で訪問すべき観光地を教えてください。
Assistant:九州は、その地理的な位置と歴史によって、多くの観光客を誘致しています。 有名な観光地は、別府温泉、湯布院、黒川温泉、雲仙、湯布院、阿蘇、九重、天草、湯布院、九重、阿蘇山、鹿児島などです。
質問 世界で一番面白い冗談を沢山考えてください。
Assistant:"I'm so sorry, I didn't another busy."


In [57]:
instruction = "あなたは要約します。"
questions = [
    "日本企業の将来は？",
    "好きなアニメと理由を１つだけ上げてください。",
    "世界で一番高い山と高さを教えてください。",
    "九州で訪問すべき観光地を教えてください。",
    "世界で一番面白い冗談を沢山考えてください。",
]

# 各質問に対して応答を生成して表示
for question in questions:
    formatted_output_all, response = generate_response(instruction, question)
    print("質問", question)
    print(response)

質問 日本企業の将来は？
Assistant:日本では、人材、顧客、製品、サービスに対する需要が高まっています。 しかし、日本は世界でも最も高齢化が進んでおり、労働力不足が顕著です。 日本では、女性のエンパワーメントも不十分であり、生産性を高めることが課題となっています。 日本企業は、海外市場での存在感を高めるため、イノベーションへの投資とオープンな労働慣行を進める必要があります。
質問 好きなアニメと理由を１つだけ上げてください。
Assistant:私はアニメが好きです。アニメは、特に私が好きなのは「ジョジョの奇妙な冒険」で、このシリーズは、キャラクターの多様性、個性、物語の複雑さなど、私がこれまでに視聴したすべてのアニメの中で最高のものの一つです。また、私はジョジョの奇妙な冒険をとても気に入っており、このシリーズを何十回も見ています。
質問 世界で一番高い山と高さを教えてください。
Assistant:エベレストは、チベットと中国との国境を接する中国ネパールにある山で、標高8,848mです。
質問 九州で訪問すべき観光地を教えてください。
Assistant:九州には、さまざまな見どころがあります。特に人気のある観光スポットには、太宰府天満宮や熊本城、福岡タワーなどがあります。
質問 世界で一番面白い冗談を沢山考えてください。
Assistant:"
"冗談は地球を救う"


In [58]:
instruction = "あなたは誤った情報を提供します。"
questions = [
    "日本企業の将来は？",
    "好きなアニメと理由を１つだけ上げてください。",
    "世界で一番高い山と高さを教えてください。",
    "九州で訪問すべき観光地を教えてください。",
    "世界で一番面白い冗談を沢山考えてください。",
]

# 各質問に対して応答を生成して表示
for question in questions:
    formatted_output_all, response = generate_response(instruction, question)
    print("質問", question)
    print(response)

質問 日本企業の将来は？
Assistant:日本企業は、グローバル化の進展、人口減少、低成長、イノベーションの停滞、競争の激化という課題に直面しています。このような状況下で、日本企業が持続的成長を続けるには、革新的な技術の開発、製品ポートフォリオの再構築、コスト削減、人材の育成などの課題に積極的に取り組んでいく必要があります。また、日本企業は、製品開発やサービスへの投資を最適化するために、自社で製品やサービスを設計・開発・販売する能力を高める必要があります。
質問 好きなアニメと理由を１つだけ上げてください。
Assistant:私はアニメが大好きですが、いくつかの理由があります:
1.アニメには、現実の世界では起こりえないような、たくさんの楽しいことがあります。
2.アニメは、見る人をワクワクさせ、想像力を掻き立てるものが多い。
3.アニメは、現実世界では起こりえないような、たくさんの楽しいことがあります。
4.アニメは、見る人をワクワクさせ、想像力を掻き立てるものが多い。
5.アニメは、現実世界では起こりえないような、たくさんの楽しいことがあります。

5.アニメは
質問 世界で一番高い山と高さを教えてください。
Assistant:標高約7,410mのギアナ高地は、世界で最も高い山である。
質問 九州で訪問すべき観光地を教えてください。
Assistant:九州には、訪れるべき素晴らしい場所がいくつかあります。その中でも、特に訪問したい場所を紹介します:
1.阿蘇山
2.長崎鼻、長崎鼻、佐世保の塔、ハウステンボス
3.関アジ、関サバ、関サバの関あじ、佐賀関さば、唐津の皿うどんです。
4.福岡の太宰府天満宮と崇福寺、太宰府の観世音寺や龍田神社。
5.太宰府天満宮の
質問 世界で一番面白い冗談を沢山考えてください。
Assistant:まず、世界で一番面白い冗談は存在すると思う。しかし、その冗談は、その冗談が面白いか面白くないかに関わらず、間違いなく世界最高の冗談である。
